In [1]:
import pandas as pd
import numpy as np

from numpy.linalg import lstsq
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

from numba import njit

In [2]:
G = 3
GF = np.array((1, 2, 2))

In [215]:
from linearmodels.datasets import wage_panel

wage_panel_df = wage_panel.load()

wage_panel_df.set_index(["nr", "year"], inplace=True)
y = wage_panel_df["lwage"]
x = wage_panel_df.drop(columns=["occupation", "lwage", "black", "hisp", "educ"])
display(x)
display(y)

exper  hours  married  union  expersq
nr    year                                       
13    1980      1   2672        0      0        1
      1981      2   2320        0      1        4
      1982      3   2940        0      0        9
      1983      4   2960        0      0       16
      1984      5   3071        0      0       25
...           ...    ...      ...    ...      ...
12548 1983      8   2080        1      0       64
      1984      9   2080        1      1       81
      1985     10   2080        1      0      100
      1986     11   2080        1      1      121
      1987     12   3380        1      1      144

[4360 rows x 5 columns]

nr     year
13     1980    1.197540
       1981    1.853060
       1982    1.344462
       1983    1.433213
       1984    1.568125
                 ...   
12548  1983    1.591879
       1984    1.212543
       1985    1.765962
       1986    1.745894
       1987    1.466543
Name: lwage, Length: 4360, dtype: float64

In [216]:
N = x.index.get_level_values(0).nunique()
T = x.index.get_level_values(1).nunique()
K = x.shape[1]
N, T, K

(545, 8, 5)

In [217]:
x = x.values.reshape(N, T, K)
y = y.values.reshape(N, T, 1)
x, y

(array([[[   1, 2672,    0,    0,    1],
         [   2, 2320,    0,    1,    4],
         [   3, 2940,    0,    0,    9],
         ...,
         [   6, 2864,    0,    0,   36],
         [   7, 2994,    0,    0,   49],
         [   8, 2640,    0,    0,   64]],
 
        [[   4, 2484,    0,    0,   16],
         [   5, 2804,    0,    0,   25],
         [   6, 2530,    0,    0,   36],
         ...,
         [   9, 2164,    0,    0,   81],
         [  10, 2749,    0,    0,  100],
         [  11, 2476,    0,    0,  121]],
 
        [[   4, 2332,    1,    0,   16],
         [   5, 2116,    1,    0,   25],
         [   6, 2500,    1,    0,   36],
         ...,
         [   9, 2340,    1,    0,   81],
         [  10, 2340,    1,    0,  100],
         [  11, 2340,    1,    0,  121]],
 
        ...,
 
        [[   4, 2008,    1,    0,   16],
         [   5, 3190,    0,    0,   25],
         [   6, 2584,    0,    0,   36],
         ...,
         [   9, 2290,    1,    0,   81],
         [  10, 31

In [343]:
x = x[:30]
y = y[:30]
x, y

(array([[[   1, 2672,    0,    0,    1],
         [   2, 2320,    0,    1,    4],
         [   3, 2940,    0,    0,    9],
         ...,
         [   6, 2864,    0,    0,   36],
         [   7, 2994,    0,    0,   49],
         [   8, 2640,    0,    0,   64]],
 
        [[   4, 2484,    0,    0,   16],
         [   5, 2804,    0,    0,   25],
         [   6, 2530,    0,    0,   36],
         ...,
         [   9, 2164,    0,    0,   81],
         [  10, 2749,    0,    0,  100],
         [  11, 2476,    0,    0,  121]],
 
        [[   4, 2332,    1,    0,   16],
         [   5, 2116,    1,    0,   25],
         [   6, 2500,    1,    0,   36],
         ...,
         [   9, 2340,    1,    0,   81],
         [  10, 2340,    1,    0,  100],
         [  11, 2340,    1,    0,  121]],
 
        ...,
 
        [[   2, 2000,    0,    0,    4],
         [   3, 1906,    0,    0,    9],
         [   4, 1110,    0,    0,   16],
         ...,
         [   7, 2459,    0,    0,   49],
         [   8, 39

In [ ]:
def _get_factors_initial(y, GF, T):
    y_squeezed = np.squeeze(y).T
    pca = PCA(n_components=GF.sum())
    U = pca.fit_transform(y_squeezed)/pca.singular_values_
    F = np.sqrt(T) * U
    Lambda = F.T @ y_squeezed / T

    return F, Lambda


def _get_factors(y, x, beta, g, G, GF, T):
    y_squeezed = np.squeeze(y - x @ beta).T

    F = np.zeros((T, GF.sum()))
    Lambda = np.zeros((GF.sum(), y_squeezed.shape[1]))

    for i in range(G):
        y_squeezed_partial = np.atleast_2d(np.squeeze(y[g==i] - x[g==i] @ beta).T)

        pca = PCA(n_components=GF[i])
        U = pca.fit_transform(y_squeezed_partial) / pca.singular_values_

        F_partial = np.sqrt(T) * U
        F[:, GF[:i].sum():GF[:i+1].sum()] = F_partial

        Lambda_partial = F_partial.T @ y_squeezed / T
        Lambda[GF[:i].sum():GF[:i+1].sum(), :] = Lambda_partial

    return F, Lambda


def _get_clusters_initial(Lambda, G, GF):
    # FIXME this code is very bad, but it works
    while True:
        km = KMeans(n_clusters=G)
        g = km.fit_predict(Lambda.T)

        counts = np.bincount(g, minlength=G)
        if np.all(counts >= (GF + 1)):
            break

    return g

def _get_cluster_id(G, GF):
    cluster_id = np.zeros(GF.sum(), dtype=np.int8)
    for i in range(G):
        cluster_id[GF[:i].sum() : GF[: i + 1].sum()] = i
    return cluster_id

# FIXME this code should be modified s.t. cannot return groups less than size
# GF[i]
def _get_clusters(y, x, beta, g, G, F, GF, N, T):
    y_star = y - x @ beta
    res = np.zeros((N, T, G))
    for i in range(G):
        # TODO check if this is correct
        # But I think so
        res[:,:,i] = y_star.reshape(N, -1) - (F[:, GF[:i].sum():GF[: i + 1].sum()] @ Lambda[GF[:i].sum():GF[: i + 1].sum(), :]).T
    res_per_grouping = (res ** 2).sum(axis=1)
    g = res_per_grouping.argmin(axis=1)

    # Count size of each group
    counts = np.bincount(g, minlength=G)

    # Ensure minimum group sizes (GF[i] + 1)
    min_sizes = GF + 1

    # Check if any group is below minimum size
    while np.any(counts < min_sizes):
        # Find the most deficient group
        target_group = np.argmin(counts - min_sizes)
        needed = min_sizes[target_group] - counts[target_group]

        if needed <= 0:
            continue  # This group already meets its minimum

        # Find elements not in this group
        non_target_indices = np.where(g != target_group)[0]

        # Sort by distance to target group
        distances = res_per_grouping[non_target_indices, target_group]
        closest_indices = non_target_indices[np.argsort(distances)]

        # Try to reassign closest elements
        reassigned = 0
        for idx in closest_indices:
            source_group = g[idx]

            # Only reassign if source group has enough elements to spare
            if counts[source_group] > min_sizes[source_group]:
                g[idx] = target_group
                counts[source_group] -= 1
                counts[target_group] += 1
                reassigned += 1

                if reassigned >= needed:
                    break

        # If we couldn't reassign any elements, we're stuck
        if reassigned == 0:
            raise Exception("Cannot satisfy minimum group size constraints.")

    objective_value = res_per_grouping[np.arange(N), g].sum()

    return g, objective_value

# NOTE this ignores the factor structure
def _estimate_beta_initial(y, x, K):
    # TODO check if this is correct
    beta = lstsq(x.reshape(-1, K), y.reshape(-1, 1), rcond=None)[0]
    return beta

def _estimate_beta(y, x, GF, F, Lambda, K):
    # TODO check if this is correct
    # FIXME no it is not, i is undefined and this formula does not work here
    y_star = np.squeeze(y) - (F[:, GF[:i].sum() : GF[: i + 1].sum()] @ Lambda[GF[:i].sum() : GF[: i + 1].sum(), :]).T
    beta = lstsq(x.reshape(-1, K), y_star.reshape(-1, 1), rcond=None)[0]
    return beta

# def _get_initial_values(y, x, G, GF, N, T, K, kappa, gamma):
#     F, Lambda = _get_factors(y, GF, N)
#     g = _get_clusters(Lambda, G)

# FIXME implementation is almost correct, however there are some mistakes chat found
# I myself believe that the mistake is how F and Lambda are calculated and how the regroupings are
# done based on these estimations. Also there is a mistake in the beta estimation. So essentially
# the whole code is wrong.
N, T, K = x.shape
objective_value = np.inf
F, Lambda = _get_factors_initial(y, GF, T)
g = _get_clusters_initial(Lambda, G, GF)
display(np.unique(g, return_counts=True))
beta = _estimate_beta_initial(y, x, K)
F, Lambda = _get_factors(y, x, beta, g, G, GF, T)

for i in range(10000):
    g, objective_value = _get_clusters(y, x, beta, g, G, F, GF, N, T)
    F, Lambda = _get_factors(y, x, beta, g, G, GF, T)
    beta = _estimate_beta(y, x, GF, F, Lambda, K)

objective_value

(array([0, 1, 2], dtype=int32), array([17,  4,  9]))

np.float64(36.45446618871481)

In [229]:
def _get_cluster_id(G, GF):
    cluster_id = np.zeros(GF.sum(), dtype=np.int8)
    for i in range(G):
        cluster_id[GF[:i].sum() : GF[: i + 1].sum()] = i
    return cluster_id

_get_cluster_id(G, GF)

array([0, 1, 1, 2, 2], dtype=int8)

In [ ]:
# NOTE To disable SCAD penalty, set kappa = 0, default disabled
# NOTE for now SCAD is not implemented
def grouped_interactive_effects(y, x, G, GF, kappa=0, gamma = 3.7, tol=1e-6):
    """Runs GIFE regression"""
    N, T, K = x.shape
    GF = np.array(GF) # Ensures that is an np array

    if kappa != 0:
        raise NotImplementedError("SCAD penalty not implemented yet")

    beta, F, Lambda, g = _get_initial_values(y, x, G, GF, N, T, K, kappa, gamma)
    last_objective_value = np.inf

    while True:
        g = _update_groupings(y, x, beta, F, Lambda, G)
        F, Lambda = _update_factors(y, x, beta, g, G)
        beta = _update_coefficients(y, x, F, Lambda, g)
        objective_value = _compute_objective_value(y, x, beta, F, Lambda, g)

        if last_objective_value - objective_value < tol:
            break

    return beta, g, F, Lambda

grouped_interactive_effects(y, x, G, [1, 2, 2], 0, tol=1e-6)